In [7]:
import cv2 as cv
import numpy as np
import os
import urllib.request

In [3]:
!mkdir yolov3

mkdir: cannot create directory ‘yolov3’: File exists


In [4]:
%cd yolov3

/content/yolov3


In [23]:
# os.getcwd()
# urllib.request.urlretrieve('https://pjreddie.com/media/files/yolov3.weights','yolov3.weights')
# urllib.request.urlretrieve('https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg','yolov3.cfg')
# urllib.request.urlretrieve('https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names','coco.names')
urllib.request.urlretrieve('https://development.asia/sites/default/files/case-study/pedestrian-friendly-streets_0.jpg','../image1.jpg')


('../image1.jpg', <http.client.HTTPMessage at 0x78426311e560>)

In [17]:
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [22]:
yolo_net = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [26]:
image = cv.imread("../image1.jpg")
height, width, channels = image.shape

In [27]:
img_blob = cv.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

In [28]:
yolo_net.setInput(img_blob)
outputs = yolo_net.forward(output_layers)
outputs[0].shape

(507, 85)

In [29]:
class_ids = []
confidences = []
boxes = []
for out in outputs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence> 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [31]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors[0],colors[1],colors[2],colors[5],colors[9] = (0, 136, 255), (226, 43, 138), (255, 191, 0), (153, 255, 255), (164, 203, 255)
indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv.putText(image, label, (x, y + 30), cv.FONT_HERSHEY_PLAIN, 0.2, color, 2)

cv.imwrite("../image-det-cv.jpg",image)

True